In [1]:
from six.moves import cPickle as pickle

import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

import os as os
import fnmatch
import argparse
import pathlib
import time

import numpy as np
if "DISPLAY" not in os.environ:
    mpl.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import scipy.io as sio
import glob 

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold  # k-fold cross validation
from sklearn.preprocessing import StandardScaler  # data normalization
from sklearn.ensemble import IsolationForest  # random forest model
from sklearn.pipeline import Pipeline # model pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics, tree

%reload_ext autoreload

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rc('text', usetex=True)
plt.rc('font', family='times')

plt.rcParams.update({'font.size': 22})
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2



In [2]:
knownNames = glob.glob("../dataset/Dados_SONAR/[A-CH]*.mat")

unknownNames = glob.glob("../dataset/Dados_SONAR/[DEFG]*.mat")

X = [sio.loadmat(x)['ent_norm'] for x in knownNames]
Y = [x.split('/')[-1][0] for x in knownNames]
X2 = [sio.loadmat(x)['ent_norm'] for x in unknownNames]
# print(X)

minLengthVector = [np.shape(x)[0] for x in X]
minLengthKnown = int(np.min(minLengthVector))

minLengthVector = [np.shape(x)[0] for x in X2]
minLengthUnknown = int(np.min(minLengthVector))

minLength = np.minimum(minLengthKnown, minLengthUnknown)
print(minLength)



# le = LabelEncoder()
# target = le.fit_transform(Y)

target = np.ones(len(X))

# minLength = np.zeros(len(names))

# for i in range(len(names)):
#     x = sio.loadmat(str(names[i]))
#     y = x["ent_norm"]
#     minLength[i] = np.shape(y)[0]
# print(np.min(minLength))

29


In [3]:
xMatrix = np.stack([x[:minLength,:].reshape(-1) for x in X])

xMatrix2 = np.stack([x[:minLength,:].reshape(-1) for x in X2])

X_train, X_test, y_train, y_test = train_test_split(xMatrix, target, test_size=0.3, random_state=42)

# pca = PCA(10)
# X_train = pca.fit_transform(X_train)
# X_test = pca.fit_transform(X_test)

model = Pipeline([('pca', PCA()), ('if', IsolationForest(random_state=42, n_jobs=-1))])


In [4]:
fold = KFold(5)

In [5]:
def test(estimator, X_test, **kwargs):
    anomalies = estimator.predict(X_test)
    return np.sum(anomalies)
    

In [11]:
# f1sc = test
f1sc = metrics.make_scorer(metrics.homogeneity_score)
parameters = {'pca__n_components': [50, 100], 'if__n_estimators': [10, 50, 100], 
              'if__contamination': [0]}
gridsearchcv = GridSearchCV(model, parameters, n_jobs=3, scoring=f1sc, cv=fold)
gridsearchcv.fit(X_train, y_train)

/home/felipe/miniconda2/envs/siemens2.7/lib/python2.7/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/felipe/miniconda2/envs/siemens2.7/lib/python2.7/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/felipe/miniconda2/envs/siemens2.7/lib/python2.7/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('if', IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
        max_samples='auto', n_estimators=100, n_jobs=-1, random_state=42,
        verbose=0))]),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'pca__n_components': [50, 100], 'if__n_estimators': [10, 50, 100], 'if__contamination': [0, 0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(homogeneity_score), verbose=0)

In [14]:
gridsearchcv.grid_scores_

/home/felipe/miniconda2/envs/siemens2.7/lib/python2.7/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 1.00000, std: 0.00000, params: {'pca__n_components': 50, 'if__n_estimators': 10, 'if__contamination': 0},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 100, 'if__n_estimators': 10, 'if__contamination': 0},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 50, 'if__n_estimators': 50, 'if__contamination': 0},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 100, 'if__n_estimators': 50, 'if__contamination': 0},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 50, 'if__n_estimators': 100, 'if__contamination': 0},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 100, 'if__n_estimators': 100, 'if__contamination': 0},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 50, 'if__n_estimators': 10, 'if__contamination': 0.1},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 100, 'if__n_estimators': 10, 'if__contamination': 0.1},
 mean: 1.00000, std: 0.00000, params: {'pca__n_components': 50, 'if__n_estimators': 50

In [13]:
print(gridsearchcv.best_params_)

metrics.accuracy_score(y_test, gridsearchcv.best_estimator_.predict(X_test))
# metrics.accuracy_score(y_test, gridsearchcv.best_estimator_.predict(X_test))

# gridsearchcv.best_estimator_predict(X2)


print(gridsearchcv.best_estimator_.predict(xMatrix2))


{'pca__n_components': 50, 'if__n_estimators': 10, 'if__contamination': 0}
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
